<a href="https://colab.research.google.com/github/SoloYolo7/streamlit/blob/main/Kino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Монтируйте Google Диск к Colab
drive.mount('/content/drive')

# Задайте путь к файлу на вашем Google Диске
file_path = "/content/drive/MyDrive/kinopoisk.jsonl"

# Вы можете использовать переменную file_path для обращения к файлу


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
# Read the dataset
data = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))

# Display the first few entries to understand the dataset structure
data[:5]

In [6]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split

# Загрузка данных
file_path = '/content/drive/MyDrive/kinopoisk.jsonl'  # Укажите правильный путь к файлу
data = pd.read_json(file_path, lines=True)

# Функция для предобработки текста
def preprocess_text(text):
    # Удаление лишних символов
    text = re.sub(r'\W', ' ', str(text))
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    text = re.sub(r'^b\s+', '', text)
    text = text.lower()  # Приведение к нижнему регистру

    # Лемматизация
    stemmer = SnowballStemmer("russian")
    tokens = text.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in stopwords.words('russian')]
    text = ' '.join(stemmed_tokens)

    return text

# Предобработка текста
X = data['content'].apply(preprocess_text)
y = data['grade3']

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Векторизация текста с использованием TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

# Теперь данные готовы к обучению модели



In [8]:
from imblearn.over_sampling import SMOTE

# Создание экземпляра SMOTE и балансировка обучающего набора
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_tfidf, y_train)

# Проверка нового распределения классов
from collections import Counter
print("Распределение классов после SMOTE:", Counter(y_train_smote))



Распределение классов после SMOTE: Counter({'Good': 21810, 'Bad': 21810, 'Neutral': 21810})


In [9]:
import pandas as pd

# Создаем новый DataFrame с балансированными данными
balanced_df = pd.DataFrame({'X_train_smote': X_train_smote, 'y_train_smote': y_train_smote})

# Указываем путь и имя файла, в который вы хотите сохранить сбалансированные данные
output_file = "сбалансированные_данные.csv"

# Сохраняем DataFrame в файл CSV
balanced_df.to_csv(output_file, index=False, sep=';')  # Вы можете выбрать другой разделитель, если нужно


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import time

# Обучение логистической регрессии
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_smote, y_train_smote)

# Измерение времени предсказания на валидационном наборе
start_time = time.time()
y_val_pred = log_reg.predict(X_val_tfidf)
end_time = time.time()
prediction_time = end_time - start_time

# Вычисление F1-метрики

f1 = f1_score(y_val, y_val_pred, average='macro')

print("Время, затраченное на предсказание:", prediction_time)
print("F1-Score для логистической регрессии:", f1)


Время, затраченное на предсказание: 0.0077648162841796875
F1-Score для логистической регрессии: 0.6500895701920437


In [11]:
import joblib

# Сохранение весов модели в файл
model_weights_filename = "/content/drive/MyDrive/logistic_regression_weights.pkl"
joblib.dump(log_reg, model_weights_filename)


['/content/drive/MyDrive/logistic_regression_weights.pkl']